# Quick start guide

This quick start guide will give a (highly incomplete) overview of the *Hylleraas Software Platform* (HSP) and how to use it to perform simple calculations. As an example we will look at the single-point energy&ndash;gradient calculation of a caffeine molecule using the semi-empirical tight binding xTB method.

<a id='install_note'></a>

```{eval-rst}
.. admonition:: Note

   This guide requires the *Hylleraas Software Platform* to be installed. Please refer to :ref:`installation` for information about how to install the HSP.

   This guide also requires the xTB software to be installed. Please refer to :ref:`quantum_chemistry_codes` for information about how to install xTB.

   Normally, **all** necessary software for this tutorial may be installed by running the following commands in the terminal:

    .. code-block:: bash

        python3 -m pip install git+https://gitlab.com/hylleraasplatform/hylleraas.git

        conda create -n conda_xtb
        conda install -c conda-forge -n conda_xtb xtb
```

## General usage workflow
Usage of the HSP typically follows the following workflow:

1. **Define the molecular system**:  
&#9; Build the molecular geometry of the system you want to study. 

2. **Establish the simulation method**:   
&#9; Choose one of the available software to perform the calculation with.

    - 2a. (*Optional*) **Setup simulation options**:  
    Choose *how* to run the simulation and which (and how many) resources to use.

3. **Run the simulation**:  
&#9; Submit the calculation with the HSP to a computing resource (the local machine, or a remote cluster).

4. **Analyze the results**:  
&#9; Retrieve the results analyze them.

In [1]:
import hylleraas as hsp

### 1. Defining the system&mdash;`Molecule` objects
The first step of any chemical calculation is to define the molecular system you want to study. This is typically done by providing the molecular geometry of the system. In the HSP, this is done by providing a molecular topology and creating a `Molecule` object. A `Molecule` object is a representation of the molecular system that can be used to perform calculations. Note that the HSP `Molecule` object is not necessarily bound together in the normal chemical covalent sense, but rather a collection of atoms and their positions in space that may represent one or more *actual* chemical molecules.

The topology of the molecule can be provided in a variety of ways, but here we will generate it from a [SMILES string](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system). For an in-depth overview of the different ways to generate `Molecule` objects, see the [molecule user guide](molecule.ipynb). 

In [65]:
caffeine = hsp.Molecule("CN1C=NC2=C1C(=O)N(C(=O)N2C)C")

The molecule can be inspected after creation to verify that it was created correctly. The list of atoms and coordinates are available as `atoms` and `coordinates` attributes. and viewed in a 3D viewer to ensure that the geometry is correct.

In [78]:
print(caffeine.atoms)
print(caffeine.coordinates)
print(caffeine.units)

['C', 'N', 'C', 'N', 'C', 'C', 'C', 'O', 'N', 'C', 'O', 'N', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
[[ 3.248941e+00 -1.983300e-02 -1.814310e-01]
 [ 1.991275e+00 -7.290750e-01 -1.200000e-05]
 [ 1.901331e+00 -2.052956e+00  2.161750e-01]
 [ 5.878090e-01 -2.312110e+00  3.288030e-01]
 [-1.249870e-01 -1.179226e+00  1.866490e-01]
 [ 7.816680e-01 -1.545080e-01 -2.609800e-02]
 [ 3.814430e-01  1.153945e+00 -2.132200e-01]
 [ 1.234012e+00  2.059833e+00 -4.040730e-01]
 [-9.361010e-01  1.419828e+00 -1.841670e-01]
 [-1.843850e+00  4.255600e-01  2.378200e-02]
 [-3.058476e+00  7.536670e-01  3.731900e-02]
 [-1.426824e+00 -8.451250e-01  2.039630e-01]
 [-2.414133e+00 -1.887063e+00  4.238160e-01]
 [-1.356099e+00  2.781776e+00 -3.787700e-01]
 [ 3.192151e+00  9.862720e-01  2.864110e-01]
 [ 4.058980e+00 -5.556260e-01  3.959420e-01]
 [ 3.554363e+00  3.750000e-04 -1.238756e+00]
 [ 2.730861e+00 -2.773587e+00  2.861950e-01]
 [-3.309331e+00 -1.744555e+00 -2.176430e-01]
 [-2.796307e+00 -1.87305

In [5]:
hsp.view_molecule(caffeine);

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### 2. Establishing the simulation method&mdash;`Method` objects
The next step is to establish the simulation method that will be used to perform the calculation. The HSP provides a variety of *interfaces* to different software packages, and the method object is used to define the software and simulation method that will be used to perform the calculation.

Method objects are created by providing the name of the software package and the method that will be used. In this case, we will use the semi-empirical tight binding xTB method, which is available through the `Xtb` interface.

In [85]:
xtb_method = hsp.Xtb(
    {
        "properties": ["energy", "gradient"],  # calculate energy and gradients
    }
)


WARNING uxtbpy: Failed to retrieve number of atoms. Check input file.



The `hsp.Xtb` method interface is called with a dictionary containing options for the simulation. Here we only provide the `properties` key with `"energy"` and `"gradient"`, meaning we will calculate energies and gradients with default xTB settings.

### 2a. Setup simulation options&mdash;`compute_settings` objects
Better control over *where* and *how* the simulation is run can be achieved by providing a `compute_settings` object. This object is used to define the resources that will be used to perform the calculation, and how the technicalities of how the calculation will be performed.

The compute settings object is created by providing a dictionary with the settings that will be used.

```{eval-rst}
.. admonition:: Note

   If you used the suggested install directives in :ref:`install_note`, you will need to link the conda environment :code:`conda_xtb` to the HSP. This is done by creating a :code:`"conda"` compute settings object and providing the name of the conda environment, as is done in the following.
```
 
The `compute_settings` settings object can link to executables in different conda environments, to executables in the normal system path, or to executables on a remote server such as a SLURM supercomputer cluster. For more information about the different options, see the [compute settings user guide](compute_settings.ipynb).


In [103]:
settings_conda_xtb = hsp.create_compute_settings(
    "conda",
    conda_env="conda_xtb",
)

Using the specific `compute_settings` object with a method object is achieved by providing the `compute_settings` object when creating the method.

In [104]:
xtb_conda_method = hsp.Xtb(
    {
        "properties": ["energy", "gradient"],  # calculate energy and gradients
    },
    compute_settings=settings_conda_xtb,
)

DEBUG: creating directory /Users/mortenledum/Documents/hsp_devdoc/hylleraas/docs/src/user_guide
DEBUG: removing file /Users/mortenledum/Documents/hsp_devdoc/hylleraas/docs/src/user_guide/xtbrestart
DEBUG: removing file /Users/mortenledum/Documents/hsp_devdoc/hylleraas/docs/src/user_guide/hessian
DEBUG: removing file /Users/mortenledum/Documents/hsp_devdoc/hylleraas/docs/src/user_guide/g98.out
DEBUG: RUNNING conda run -n conda_xtb xtb --version
DEBUG: IN FOLDER /Users/mortenledum/Documents/hsp_devdoc/hylleraas/docs/src/user_guide
DEBUG: RESULTS: LocalResult(files_to_parse=[], output_file=None, add_to_results=None, stdout="      -----------------------------------------------------------      \n     |                   =====================                   |     \n     |                           x T B                           |     \n     |                   =====================                   |     \n     |                         S. Grimme                         |     \n     |

/Users/mortenledum/Documents/hsp_devdoc/hyset/build/__editable__.hyset-0.1-py3-none-any/hyset/local/runner.py:255: UserWarning: normal termination of xtb


  warnings.warn(result.stderr)


### 3. Running the simulation
The simulation is run by providing the `Molecule` object and calling the `.run()` method of the `Method` object. The `.run()` method will submit the calculation to the computing resource defined in the `compute_settings` object, and return a `Result` object.

All HSP calculations are evaluated [lazily](https://en.wikipedia.org/wiki/Lazy_evaluation), meaning only the necessary simulation will be ran and once ran, it will not be re-ran if subsequent calls to `.run()` are made (unless explicitly requested via the `force_recompute` flag).

In [102]:
caffeine_xtb_result = xtb_conda_method.run(caffeine, debug=True)

Instead of (or in addition to) the `.run(molecule)` method, you may query directly a desired quantity such as the energy. This is done by the `.get_energy(molecule)` method. Note carefully that the simulation will now **not** be re-done, as this result is already cached.

In [93]:
energy = xtb_conda_method.get_energy(caffeine)

print(energy)

-42.121970452276


### 4. Analyzing the results
The results of the simulation are stored in a `Result` object. The `Result` object contains any and all available properties resulting from the simulation ran. These properties may be accessed and inspected by querying the `Result` object like a normal dictionary.

An easy way to list of all available properties in the result object is to call the `.keys()` method of the `Result` object.

In [97]:
for key in caffeine_xtb_result.keys():
    print(key)

version
homo_energy
lumo_energy
polarisability
wiberg_index_matrix
dipole_moment
energy
homo_lumo_gap
gradient


-------------------------